In [1]:
# @title
### EP2 MAC0417 / MAC5768
##################################################################
# AO PREENCHER ESSE CABEÇALHO COM O MEU NOME E O MEU NÚMERO USP,#
# DECLARO QUE SOU O ÚNICO AUTOR E RESPONSÁVEL PELA RESOLUÇÃO #
# DESTE EP. #
# TODAS AS PARTES FORAM DESENVOLVIDAS E IMPLEMENTADAS POR MIM, #
# SEGUINDO AS INSTRUÇÕES E QUE PORTANTO, NÃO CONSTITUEM #
# DESONESTIDADE ACADÊMICA OU PLÁGIO. #
# #
# DECLARO TAMBÉM, QUE SOU RESPONSÁVEL POR TODAS AS CÓPIAS #
# DESSE PROGRAMA, E QUE EU NÃO DISTRIBUI OU FACILITEI A #
# SUA DISTRIBUIÇÃO. ESTOU CIENTE QUE OS CASOS DE PLÁGIO E #
# DESONESTIDADE ACADÊMICA SERÃO TRATADOS SEGUNDO OS CRITÉRIOS #
# DEFINIDOS NO CÓDIGO DE ÉTICA DA USP. #
# #
# ENTENDO QUE JUPYTER NOTEBOOKS SEM ASSINATURA NÃO SERÃO #
# CORRIGIDOS E, AINDA ASSIM, PODERÃO SER PUNIDOS POR #
# DESONESTIDADE ACADÊMICA. #
# #
# #
# Nome : Bruna Luiza Bispo
# NUSP : 11488516
# Turma: MAC5768 - Visão e Processamento de Imagens (2024)
# Prof.: Ronaldo Fumio Hashimoto
##################################################################

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
from cv2 import imwrite

In [3]:
#Caminhos EP2
#originalGrayDataset
#augmentedDataset_expo
#augmentedDataset_laplaciano
#augmentedDataset_log
#augmentedDataset_media
#augmentedDataset_stretching
#normalizedDataset_expo
#normalizedDataset_laplaciano
#normalizedDataset_log
#normalizedDataset_media
#normalizedDataset_stretching

In [4]:
#Parametros
kernel_size = 3
sigma = 2

In [5]:
def gaussian_kernel(size, sigma=1):
    kernel = np.fromfunction(
        lambda x, y: (1/ (2 * np.pi * sigma**2)) * np.exp( - ((x - (size-1)/2)**2 + (y - (size-1)/2)**2) / (2 * sigma**2)), 
        (size, size)
    )
    return kernel / np.sum(kernel)

gaussian_kernel = gaussian_kernel(kernel_size, sigma)

def convolucao(image, kernel):
    kernel_height, kernel_width = kernel.shape
    image_height, image_width = image.shape
    
    pad_height = kernel_height // 2
    pad_width = kernel_width // 2
    
    padded_image = np.pad(image, ((pad_height, pad_height), (pad_width, pad_width)), mode='constant')
    
    new_image = np.zeros_like(image)
    
    for i in range(image_height):
        for j in range(image_width):
            new_image[i, j] = np.sum(padded_image[i:i+kernel_height, j:j+kernel_width] * kernel)
    
    return new_image

In [6]:
for nome_pasta in os.listdir():
    if os.path.isdir(nome_pasta) and nome_pasta != '.ipynb_checkpoints':
        caminho_gaussiano = nome_pasta + '_gaussiano'
        if not os.path.isdir(caminho_gaussiano):
            os.mkdir(caminho_gaussiano)
        for nome_arquivo in os.listdir(nome_pasta):
            if ".jpeg" in nome_arquivo:
                imagem_cinza = plt.imread(os.path.join(nome_pasta, nome_arquivo))
                imagem_gaussiana = convolucao(imagem_cinza, gaussian_kernel)
                imwrite(os.path.join(caminho_gaussiano, nome_arquivo), imagem_gaussiana)

In [7]:
def media(imagem, kernel_size):  
    image_height, image_width = imagem.shape
    
    pad_height = kernel_size // 2
    pad_width = kernel_size // 2
    padded_image = np.pad(imagem, ((pad_height, pad_height), (pad_width, pad_width)), mode='constant')
    
    # Nova imagem
    new_image = np.zeros_like(imagem)
    
    # Aplicar o filtro de mediana
    for i in range(image_height):
        for j in range(image_width):
            # Extrair a subjanela
            sub_window = padded_image[i:i+kernel_size, j:j+kernel_size]
            # Calcular a mediana e atribuir ao pixel central
            new_image[i, j] = np.median(sub_window)
    
    return new_image

In [ ]:
for nome_pasta in os.listdir():
    if os.path.isdir(nome_pasta) and nome_pasta != '.ipynb_checkpoints':
        caminho_media = nome_pasta + '_media'
        if not os.path.isdir(caminho_media):
            os.mkdir(caminho_media)
        for nome_arquivo in os.listdir(nome_pasta):
            if ".jpeg" in nome_arquivo:
                imagem_cinza = plt.imread(os.path.join(nome_pasta, nome_arquivo))
                imagem_media = media(imagem_cinza, kernel_size)
                imwrite(os.path.join(caminho_media, nome_arquivo), imagem_media)

In [ ]:
def compute_gradient(image):
    # Kernels de Sobel
    Kx = np.array([[-1, 0, 1], [-2, 0, 2], [-1, 0, 1]])
    Ky = np.array([[-1, -2, -1], [0, 0, 0], [1, 2, 1]])
    
    # Obter dimensões da imagem
    height, width = image.shape
    
    # Aplicar padding
    padded_image = np.pad(image, ((1, 1), (1, 1)), mode='constant')
    
    # Gradientes
    Gx = np.zeros_like(image)
    Gy = np.zeros_like(image)
    
    for i in range(1, height+1):
        for j in range(1, width+1):
            region = padded_image[i-1:i+2, j-1:j+2]
            Gx[i-1, j-1] = np.sum(region * Kx)
            Gy[i-1, j-1] = np.sum(region * Ky)
    
    # Magnitude do gradiente
    magnitude = np.sqrt(Gx**2 + Gy**2)
    
    return magnitude

def apply_threshold(image, threshold):
    binary_image = np.zeros_like(image)
    binary_image[image > threshold] = 255
    return binary_image

In [ ]:
for nome_pasta in os.listdir():
    if 'gaussiano' in nome_pasta:
        for nome_arquivo in os.listdir(nome_pasta):
            caminho_threshold = nome_pasta + '_threshold'
            if not os.path.isdir(caminho_threshold):
                os.mkdir(caminho_threshold)
            imagem_cinza = plt.imread(os.path.join(nome_pasta, nome_arquivo))
            gradient_magnitude = compute_gradient(imagem_cinza)
            threshold_value = 10
            binary_image = apply_threshold(gradient_magnitude, threshold_value)
            imwrite(os.path.join(caminho_threshold, nome_arquivo), binary_image)